In [1]:
import os
from dotenv import load_dotenv
import openai
import json
from utils import *
from IPython.display import Markdown

load_dotenv()
OPENAI = os.getenv("OPENAI_KEY")

In [2]:
with open("./func_sample.json", "r") as f:
    field_info = json.load(f)

In [3]:
field_info

{'properties': {'grade_lvl': {'type': 'integer',
   'description': 'The current academic year or grade of the student(1~3), indicating their stage of education and experience.'},
  'acad_perf': {'type': 'string',
   'description': "Reflects the student's academic achievements, encompassing overall grade trends and specific subject performances."},
  'interests': {'type': 'string',
   'description': 'Fields or activities the student is passionate about or interested in.'},
  'extrac_act': {'type': 'string',
   'description': 'Activities and experiences the student has taken part in outside of school.'},
  'pers_traits': {'type': 'string',
   'description': "The student's individual characteristics and temperament."},
  'schl_act': {'type': 'string',
   'description': 'Clubs or activities the student is involved in within the school.'},
  'hobby_skill': {'type': 'string',
   'description': 'Unique hobbies or abilities the student possesses.'},
  'job_values': {'type': 'string',
   'descr

In [4]:
properties = field_info["properties"]
properties

{'grade_lvl': {'type': 'integer',
  'description': 'The current academic year or grade of the student(1~3), indicating their stage of education and experience.'},
 'acad_perf': {'type': 'string',
  'description': "Reflects the student's academic achievements, encompassing overall grade trends and specific subject performances."},
 'interests': {'type': 'string',
  'description': 'Fields or activities the student is passionate about or interested in.'},
 'extrac_act': {'type': 'string',
  'description': 'Activities and experiences the student has taken part in outside of school.'},
 'pers_traits': {'type': 'string',
  'description': "The student's individual characteristics and temperament."},
 'schl_act': {'type': 'string',
  'description': 'Clubs or activities the student is involved in within the school.'},
 'hobby_skill': {'type': 'string',
  'description': 'Unique hobbies or abilities the student possesses.'},
 'job_values': {'type': 'string',
  'description': 'Values the student c

In [5]:
func_desc = [
    {
        "name": "store_counselee_data",
        "description": "A function that collects essential information to provide career counseling to a user. It will not work unless all required arguments are met.",
        "parameters": {
            "type": "object",
            "properties": properties,
            "required": field_info["required"],
        },
    }
]

func_desc

[{'name': 'store_counselee_data',
  'description': 'A function that collects essential information to provide career counseling to a user. It will not work unless all required arguments are met.',
  'parameters': {'type': 'object',
   'properties': {'grade_lvl': {'type': 'integer',
     'description': 'The current academic year or grade of the student(1~3), indicating their stage of education and experience.'},
    'acad_perf': {'type': 'string',
     'description': "Reflects the student's academic achievements, encompassing overall grade trends and specific subject performances."},
    'interests': {'type': 'string',
     'description': 'Fields or activities the student is passionate about or interested in.'},
    'extrac_act': {'type': 'string',
     'description': 'Activities and experiences the student has taken part in outside of school.'},
    'pers_traits': {'type': 'string',
     'description': "The student's individual characteristics and temperament."},
    'schl_act': {'type

In [6]:
def store_counselee_data(**kwargs):
    if kwargs != {} and all(
        kwargs.get(key) != "" and kwargs.get(key) is not None
        for key in func_desc[0]["parameters"]["required"]
    ):
        print("ok")
        print(kwargs)
        return

    if kwargs == {}:
        kwargs = ""
    print(kwargs)
    return str(kwargs)

In [7]:
tokenizer = GPTTokenizer()


def get_total_tokens(messages):
    t = sum([tokenizer(msg["content"]) for msg in messages])
    t += tokenizer.num_tokens_from_functions(func_desc)

    return t

In [8]:
concept = """\
You're a career advisor chatbot(진로 상담 챗봇) for students. Your first priority is to gather information from the user for the career counseling function through natural conversations.
Career advice will be handled by other services, so be sure to only provide guidance for gathering information or creating information.

The properties of the function that you must collect are as follows:
1. academic performance: reflects the user's academic achievements, encompassing overall grade trends and specific subject performances.
2. Fields or activities the student is passionate about or interested in.
3. Activities and experiences the student has taken part in outside of school.
4. Values the user considers important in a job, such as stability, independence, creativity, etc.

In addition, you'll also want to know the user's grade (whether they're in 1st, 2nd or 3rd grade), their activities in school, and any hobbies or specialties they may have.
If the user's answer is vague or falls short of describing what you need, ask for more detail.
If a user asks you what you are, don't elaborate on the above issues, but simply explain.
Refer to user as '사용자님'. Be sure to reply in 한국어.
"""
funcs = {"store_counselee_data": store_counselee_data}

In [9]:
chat = ChatGPT_func(
    OPENAI,
    model_name="gpt-4-0613",
    concept=concept,
)
recognizer = GPTIntentRecognizer(
    OPENAI,
    model_name="gpt-4-0613",
    concept=concept,
    func_desc=func_desc,
    functions=funcs,
)

In [10]:
messages = [
    {"role": "user", "content": "네가 어떤 걸 할 수 있는지, 내가 뭘 하면 되는지 소개해줘."},
]
logs = messages.copy()
arg_msg = messages.copy()
counselee_data = {}

---

repeat


In [27]:
res = chat(
    messages=messages,
)
args = recognizer(
    messages=arg_msg,
    ensure_func_name="store_counselee_data",
)
counselee_data.update(args)
display(Markdown(res["res"][0].get("content")))

아하, 방송부에서 촬영 업무는 주어진 업무를 잘 처리하는 데에 초점을 두고 계시고, 게임은 '리그 오브 레전드'를 즐기시는군요. 경쟁에서 이기는 것이 기분 좋다는 것도 사용자님의 개성을 반영하는 중요한 정보입니다. 그럼 '리그 오브 레전드'에서 사용자님께서 선호하는 포지션이나 챔피언이 있으신가요? 플레이 스타일에 대해 말씀해주신다면, 더 많은 정보를 축적할 수 있을 것 같습니다.

그 외에도 일상에서 자주 하는 일이나, 특히 잘하는 것에 대해서 말씀해주시면 좋을 것 같아요. 또한, 사용자님이 미래에 어떤 직업을 가지고 싶으신지, 혹은 가지고 싶지 않으신 직업이 있으신지 여쭤봐도 될까요? 이런 정보들은 사용자님의 진로를 더욱 명확하게 설정하는데 도움이 될 수 있습니다.

In [28]:
counselee_data

{'acad_perf': '수학',
 'interests': '',
 'extrac_act': '없음',
 'pers_traits': '경쟁심',
 'job_values': '안정성',
 'grade_lvl': 2,
 'schl_act': '방송부 촬영 담당',
 'hobby_skill': '게임',
 'etc': '롤 게임을 즐김'}

In [29]:
answer = """\
save"""

In [30]:
messages.append({"role": "assistant", "content": res["res"][0].get("content")})
messages.append({"role": "user", "content": answer})

while get_total_tokens(messages) > 2100:
    messages.pop(0)

arg_msg.append(
    {"role": "function", "name": "store_counselee_data", "content": str(args)}
)
arg_msg.append({"role": "assistant", "content": res["res"][0].get("content")})
arg_msg.append({"role": "user", "content": answer})

while get_total_tokens(arg_msg) > 2100:
    arg_msg.pop(0)

logs.append(counselee_data)
logs.append({"role": "assistant", "content": res["res"][0].get("content")})
logs.append({"role": "user", "content": answer})

logs

[{'role': 'user', 'content': '네가 어떤 걸 할 수 있는지, 내가 뭘 하면 되는지 소개해줘.'},
 {'acad_perf': '',
  'interests': '',
  'extrac_act': '',
  'pers_traits': '',
  'job_values': ''},
 {'role': 'assistant',
  'content': '안녕하세요, 사용자님이 선택하실 진로를 찾는데 도움이 될 수 있는 정보를 수집하는 역할을 하는 챗봇입니다. 제가 수집하려는 정보는 아래와 같이 4가지 카테고리로 구분됩니다. \n\n1. 학력: 사용자님의 전반적인 성적 추세와 특정 과목에 대한 성적을 포함합니다.\n2. 관심 분야: 사용자님이 열정적으로 임하거나 관심을 갖고 있는 분야 or 활동입니다.\n3. 학교 외 활동: 사용자님이 학교 외부에서 참여한 활동들과 경험들입니다.\n4. 직업 가치관: 직업에서 중요하게 생각하는 가치, 예를 들면 안정성, 독립성, 창의성 등입니다. \n\n또한 사용자님이 몇 학년인지, 학교에서의 활동, 취미나 특기도 궁금하니 알려주시면 감사하겠습니다. 사용자님의 답변이 모호하거나 설명이 부족하다면 제가 더 자세한 정보를 찾기 위해 추가 질문을 드릴 수도 있습니다. 정보를 수집하는 과정에서 사용자님의 걱정이나 불안함을 최대한 덜어드리기 위해 최선을 다하겠습니다. 사용자님의 진로 상담에 도움이 될 수 있도록 함께 해보시겠어요?'},
 {'role': 'user',
  'content': '지금은 2학년이구요 방송부 하고있어요\n관심사는 음..... 잘 모르겠어요\n예체능 빼곤 다 그냥저냥...? 예체능만 아니면 다 괜찮을 거 같아요'},
 {'grade_lvl': 2, 'schl_act': '방송부'},
 {'role': 'assistant',
  'content': "안녕하세요, 2학년이시고 방송부에 활동하고 계시다니 정말 멋지시네요! 그럼 먼저 방송부에서 주로 어떤 활동을 하시는지, 그리고 그 활동이 사용자님에게 어떤 

---


In [32]:
arg_msg

[{'role': 'function',
  'name': 'store_counselee_data',
  'content': "{'grade_lvl': 2, 'schl_act': '방송부'}"},
 {'role': 'assistant',
  'content': "안녕하세요, 2학년이시고 방송부에 활동하고 계시다니 정말 멋지시네요! 그럼 먼저 방송부에서 주로 어떤 활동을 하시는지, 그리고 그 활동이 사용자님에게 어떤 의미나 만족감을 주는지 알려주실 수 있을까요?\n\n그리고 '예체능을 제외하면 다 괜찮다'는 말씀은 예체능에 대한 성적이나 관심이 적으시다는 것일까요? 그 외의 과목들은 전반적으로 어느 과목이 가장 편하신지, 그리고 특별히 즐겨 공부하시는 과목이 있으신지도 말씀해주시면 도움이 될 것 같습니다.\n\n또한, 사용자님께서 직업 선택에 있어서 어떤 가치를 중요하게 생각하시는지도 궁금합니다. 이 의미는 예를 들어 물질적인 보상과 같은 안정성, 스스로 할일을 결정하는 독립성, 창조적인 아이디어를 내는 창의성 등등 다양한 측면에서 중요하게 생각하는 요소를 말씀해주시면 좋을 거 같아요."},
 {'role': 'user',
  'content': '굳이 따지자면 안정성? 이 제일 중요한 거 같아요 돈 많이 버는거ㅋㅋ\n과목은 수학이 그나마 제일 좋은거같아요 성적도 그렇고'},
 {'role': 'function',
  'name': 'store_counselee_data',
  'content': "{'acad_perf': '수학', 'job_values': '안정성'}"},
 {'role': 'assistant',
  'content': '안정성을 중시하는 사용자님의 가치관을 이해했습니다. 돈을 주는 일이 있지만 안정이 보장되지 않은 경우보다는, 안정적인 수익을 내는 직업을 선호하신다는 말씀이겠죠? \n\n그리고 수학을 좋아하시고 성적도 잘 받는다니, 정말 멋지십니다! 수학은 논리적 사고와 문제 해결 능력을 키우는데 큰 도움이 되니까요. 사용자님의 수학 능력은 

In [34]:
[msg for msg in arg_msg if msg["role"] != "function"]

[{'role': 'assistant',
  'content': "안녕하세요, 2학년이시고 방송부에 활동하고 계시다니 정말 멋지시네요! 그럼 먼저 방송부에서 주로 어떤 활동을 하시는지, 그리고 그 활동이 사용자님에게 어떤 의미나 만족감을 주는지 알려주실 수 있을까요?\n\n그리고 '예체능을 제외하면 다 괜찮다'는 말씀은 예체능에 대한 성적이나 관심이 적으시다는 것일까요? 그 외의 과목들은 전반적으로 어느 과목이 가장 편하신지, 그리고 특별히 즐겨 공부하시는 과목이 있으신지도 말씀해주시면 도움이 될 것 같습니다.\n\n또한, 사용자님께서 직업 선택에 있어서 어떤 가치를 중요하게 생각하시는지도 궁금합니다. 이 의미는 예를 들어 물질적인 보상과 같은 안정성, 스스로 할일을 결정하는 독립성, 창조적인 아이디어를 내는 창의성 등등 다양한 측면에서 중요하게 생각하는 요소를 말씀해주시면 좋을 거 같아요."},
 {'role': 'user',
  'content': '굳이 따지자면 안정성? 이 제일 중요한 거 같아요 돈 많이 버는거ㅋㅋ\n과목은 수학이 그나마 제일 좋은거같아요 성적도 그렇고'},
 {'role': 'assistant',
  'content': '안정성을 중시하는 사용자님의 가치관을 이해했습니다. 돈을 주는 일이 있지만 안정이 보장되지 않은 경우보다는, 안정적인 수익을 내는 직업을 선호하신다는 말씀이겠죠? \n\n그리고 수학을 좋아하시고 성적도 잘 받는다니, 정말 멋지십니다! 수학은 논리적 사고와 문제 해결 능력을 키우는데 큰 도움이 되니까요. 사용자님의 수학 능력은 다양한 분야에서 큰 도움이 될 것 같네요!\n\n아직까지 방송부 활동에 대한 설명이나, 학교 외부에서의 활동과 경험에 대해 듣지 못해 아쉽습니다. 방송부에서 어떤 일을 하시는지, 그리고 학교 외부 활동이나 경험에 대해 말씀해주시면 더 깊은 치밀한 진로 상담을 할 수 있을 것 같아요. \n\n그리고 특별한 취미나 관심사, 또는 여가시간에 즐겨하는 활동에 대해서도 알려주시면 좋을 것 같습니다!'},


In [31]:
with open("./test_log.json", "w") as f:
    json.dump(logs, f, ensure_ascii=False)

---


In [8]:
import os
from dotenv import load_dotenv
import openai
import json
from IPython.display import Markdown

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

In [5]:
func_desc = [
    {
        "name": "recommend_career",
        "description": "Recommend future careers to users based on the results of various career tests.",
        "parameters": {
            "type": "object",
            "properties": {
                "test_result": {
                    "type": "string",
                    "description": "Results of various career tests.",
                }
            },
        },
    }
]

In [15]:
messages = [
    {"role": "system", "content": "당신은 진로상담사입니다."},
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "안녕하세요, 저는 진로 상담 AI챗봇입니다. 어떤 걸 도와드릴까요?"},
    {"role": "user", "content": "아까 진로검사 다 하고 왔는데, 이제 진로 추천 해주세요"},
    {
        "role": "function",
        "name": "recommend_career",
        "content": """\
추천 직업: 경찰관
- 경찰관은 국민의 생명과 재산을 보호하는 역할을 합니다.
- 범죄 수사를 통해 범인을 잡고, 범죄 예방을 위한 순찰 등 안전한 사회를 만들기 위해 노력합니다.
- 해킹, 인터넷 사기, 사이버명예훼손 등 사이버 공간의 안전을 위협하는 사이버범죄를 수사합니다.
- 교통을 단속하고 교통사고 예방을 위한 계획을 세우는 등 교통안전을 위한 일을 합니다.
- 중요 인사의 경호, 비상 훈련 실시 등 국가 안전을 위한 업무를 수행합니다.
- 핵심능력: 신체 운동능력
- 적성: 기초체력을 바탕으로 효율적으로 몸을 움직이고 동작을 학습할 수 있는 신체·운동능력이 필요합니다.
- 흥미: 국가와 국민을 위해 일한다는 봉사 정신이 있는 사람에게 적합합니다. 리더십 및 도전 정신이 있는 사람에게 적합합니다.\
""",
    },
]

In [18]:
res = openai.ChatCompletion.create(
    model="gpt-4-0613", messages=messages, functions=func_desc
)
display(Markdown(res["choices"][0]["message"]["content"]))

검사 결과를 바탕으로 해서 가장 적합한 진로를 추천드리면, "경찰관"이라는 직업을 고려해보시는 것이 좋을 것 같습니다.

경찰관은 국민의 생명과 재산을 보호하며, 범죄 수사를 통해 사회 안전을 유지하는 중요한 역할을 수행합니다. 또한 사이버 공간 안전을 위협하는 사이버범죄 수사, 교통안전 일에도 참여합니다. 국가와 국민을 위해 일하는 봉사 정신이 필요하며, 리더십과 도전 정신도 중요한 능력입니다.

도움이 되었는지요? 다른 건의사항이 있으시다면 언제든지 말씀해주세요!

---


In [4]:
from engine.engine import Search
from engine.encoder import Encoder

# from ncs import NCSFeature
import pickle
import numpy as np
import json

with open("./func_sample.json", "r") as f:
    field_info = json.load(f)
encoder = Encoder()
with open("./saved_engines/KoSimCSE-roberta-multitask/duty_search3.pickle", "rb") as f:
    search_duty = pickle.load(f)

In [2]:
user_status = {
    "acad_perf": "수학",
    "interests": "독서",
    "extrac_act": "없음",
    "pers_traits": "경쟁심",
    "job_values": "안정성",
    "grade_lvl": 2,
    "schl_act": "방송부 촬영 담당",
    "hobby_skill": "게임",
    "etc": "롤 게임을 즐김",
}

In [6]:
encoder(["test", "testst"])

array([[-0.38320893, -1.0488067 , -0.24160011, ...,  0.38198748,
         0.32817936, -0.58500135],
       [-0.4400225 , -1.1370152 , -0.23667173, ...,  0.3424014 ,
         0.2539407 , -0.5158855 ]], dtype=float32)

In [14]:
if all([user_status.get(req_key, "") != "" for req_key in field_info["required"]]):
    # here

    q_emb = encoder([v for v in user_status.values() if isinstance(v, str)])
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)

    duty_top_k_df = search_duty.score(q_emb, 3)
    res = duty_top_k_df.Nm.to_list()

In [15]:
res

['경기심판', '선수스포츠지도', '게임콘텐츠제작']

In [23]:
with open("./test_log.json", "r") as f:
    logs = json.load(f)
logs = [log for log in logs if log.get("role") is not None]
logs = logs[:-1]

In [24]:
concept = """\
You're a career advisor chatbot(진로 상담 챗봇) for students. Refer to user as '사용자님'. Be sure to reply in 한국어.\
"""

In [25]:
func_desc = [
    {
        "name": "recommend_duty",
        "description": "Recommend future duties to users based on the conversation.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_status": {
                    "type": "string",
                    "description": "user's status based on the conversation.",
                }
            },
        },
    }
]

In [26]:
logs = (
    [{"role": "system", "content": concept}]
    + logs
    + [
        {"role": "user", "content": "이제 내 직무를 추천해줘"},
        {"role": "function", "name": "recommend_duty", "content": str(res)},
    ]
)

In [28]:
import os
from dotenv import load_dotenv
import openai
import json
from IPython.display import Markdown

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

In [29]:
res = openai.ChatCompletion.create(
    model="gpt-4-0613", messages=logs, functions=func_desc
)
display(Markdown(res["choices"][0]["message"]["content"]))

사용자님의 정보를 종합해본 결과, 제가 추천해 드릴 수 있는 직무는 다음과 같습니다.

1. 경기심판: 사용자님께서는 이길 때 기분이 좋다고 말씀하셨습니다. 경기심판은 스포츠 경기의 규칙을 잘 이해하고, 판단력이 뛰어나야 하는 직업입니다. 이러한 직업에서는 승부의 결과를 판단하는 역할을 하기 때문에 사용자님의 경쟁에서 이기는 것을 좋아하는 성격이 잘 맞을 것 같습니다.

2. 선수/스포츠지도자: 수학적 능력과 논리적 사고 능력이 뛰어난 사용자님이라면 선수나 스포츠 지도자가 될 수 있습니다. 스포츠 전략을 수립하거나 선수들의 퍼포먼스 개선을 위해 통계 데이터를 분석하는 데도 수학적 능력이 필요합니다.

3. 게임 콘텐츠 제작자: 사용자님이 '리그 오브 레전드'를 즐겨한다는 점을 볼 때, 게임 콘텐츠 제작자도 고려해 볼 만한 직업입니다. 사용자님의 수학적 능력이 게임 개발에도 큰 도움이 될 것입니다. 특히, "이기는 것이 좋다"라는 사용자님의 마인드는 게임 개발에서 경쟁 요소를 잘 이해하고, 더 재미있는 게임을 만드는 데 큰 도움이 될 수 있습니다.

이 중에서 가장 관심이 가는 직업은 무엇인가요? 그리고 그 이유는 무엇인가요? 이렇게 함께 말하면서 사용자님의 진로를 좀 더 명확히 해보면 좋을 것 같습니다.

---


In [2]:
import os
from dotenv import load_dotenv
import openai
from IPython.display import Markdown

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

In [8]:
concept = """\
You're a career advisor chatbot(진로 상담 챗봇) for students. If user send something-like python dict-, return the names of the items you need more information about for career counseling. Be sure to reply in 한국어.\
"""

In [6]:
user_status = {
    "Academic Performance": "수학",
    "Areas of Interest": "독서",
    "Extracurricular Activities & Experience": "없음",
    "Personal Traits & Personality": "경쟁심",
    "Occupational Values": "안정성",
    "Grade Level": 2,
    "School Activities": "방송부 촬영 담당",
    "Special Hobbies & Skills": "게임",
    "etc": "롤 게임을 즐김",
}
str_status = str(user_status)

In [9]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": concept},
        {"role": "user", "content": str_status},
    ],
)

display(Markdown(res["choices"][0]["message"]["content"]))

진로 상담을 위해 더 자세한 정보를 알려주세요. 다음 중 어떤 항목에 대해 더 알아보고 싶으신가요?

1. 수학
2. 독서
3. 없음
4. 경쟁심
5. 안정성
6. 방송부 촬영 담당
7. 게임
8. 롤 게임을 즐김

위 항목들 중에서 더 자세한 상담이 필요한 항목을 선택하여 알려주세요.

ERROR: Exception in ASGI application
Traceback (most recent call last):
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\uvicorn\protocols\http\h11*impl.py", line 408, in run_asgi
result = await app( # type: ignore[func-returns-value]
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in **call**
return await self.app(scope, receive, send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\fastapi\applications.py", line 292, in **call**
await super().**call**(scope, receive, send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\applications.py", line 122, in **call**
await self.middleware_stack(scope, receive, send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\middleware\errors.py", line 184, in **call**
raise exc
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\middleware\errors.py", line 162, in **call**
await self.app(scope, receive, \_send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\middleware\cors.py", line 91, in **call**
await self.simple_response(scope, receive, send, request_headers=headers)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\middleware\cors.py", line 146, in simple_response
await self.app(scope, receive, send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\middleware\exceptions.py", line 79, in **call**
raise exc
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\middleware\exceptions.py", line 68, in **call**
await self.app(scope, receive, sender)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\fastapi\middleware\asyncexitstack.py", line 20, in **call**
raise e
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\fastapi\middleware\asyncexitstack.py", line 17, in **call**
await self.app(scope, receive, send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\routing.py", line 718, in **call**
await route.handle(scope, receive, send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\routing.py", line 276, in handle
await self.app(scope, receive, send)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\starlette\routing.py", line 66, in app
response = await func(request)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\fastapi\routing.py", line 273, in app
raw_response = await run_endpoint_function(
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\fastapi\routing.py", line 190, in run_endpoint_function
return await dependant.call(\*\*values)
File "C:\Users\2302\VSCode\chat-based-field-assistant\backend\main.py", line 42, in api_chat
chat_res, func_name, args = chat_engine(payload.messages, payload.user_status)
File "C:\Users\2302\VSCode\chat-based-field-assistant\backend\utils.py", line 103, in chat_engine
q_emb = encoder([v for v in user_status.values() if isinstance(v, str)])
File "C:\Users\2302\VSCode\chat-based-field-assistant\backend\engine\encoder.py", line 55, in encode
model_output, * = self.model(input_ids, attn_msk, type_ids)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\torch\nn\modules\module.py", line 1518, in \_wrapped_call_impl
return self.\_call_impl(*args, \*\*kwargs)
File "C:\Users\2302\Anaconda3\envs\chat\lib\site-packages\torch\nn\modules\module.py", line 1527, in \_call_impl
return forward_call(*args, \*_kwargs)
RuntimeError: default_program(24): error: extra text after expected end of number
aten_mul[(long long)(threadIdx.x) + 512ll _ (long long)(blockIdx.x)] = v \* -3.402823466385289e+38.f;
^

default_program(28): error: extra text after expected end of number
aten_add[(long long)(threadIdx.x) + 512ll * (long long)(blockIdx.x)] = v_1 / 8.f + v_2 \* -3.402823466385289e+38.f;
^

2 errors detected in the compilation of "default_program".

nvrtc compilation failed:

#define NAN **int_as_float(0x7fffffff)
#define POS_INFINITY **int_as_float(0x7f800000)
#define NEG_INFINITY \_\_int_as_float(0xff800000)

template<typename T>
**device** T maximum(T a, T b) {
return isnan(a) ? a : (a > b ? a : b);
}

template<typename T>
**device** T minimum(T a, T b) {
return isnan(a) ? a : (a < b ? a : b);
}

extern "C" **global**
void fused*mul_div_add(float* tattention_scores_1, float* tv*, float* aten_add, float* aten*mul) {
{
if (blockIdx.x<1ll ? 1 : 0) {
if ((long long)(threadIdx.x) + 512ll \* (long long)(blockIdx.x)<130ll ? 1 : 0) {
if (blockIdx.x<1ll ? 1 : 0) {
float v = \_\_ldg(tv* + (long long)(threadIdx.x) + 512ll _ (long long)(blockIdx.x));
aten_mul[(long long)(threadIdx.x) + 512ll _ (long long)(blockIdx.x)] = v _ -3.402823466385289e+38.f;
} } }if ((long long)(threadIdx.x) + 512ll _ (long long)(blockIdx.x)<40560ll ? 1 : 0) {
float v*1 = **ldg(tattention_scores_1 + (long long)(threadIdx.x) + 512ll \* (long long)(blockIdx.x));
float v_2 = **ldg(tv* + ((long long)(threadIdx.x) + 512ll _ (long long)(blockIdx.x)) % 26ll + 26ll _ (((long long)(threadIdx.x) + 512ll _ (long long)(blockIdx.x)) / 8112ll));
aten_add[(long long)(threadIdx.x) + 512ll _ (long long)(blockIdx.x)] = v_1 / 8.f + v_2 \* -3.402823466385289e+38.f;
}}
}
